In [1]:
def get_flow_value(cur_solution: list, dist: list, flow: list):
    value = 0
    for i in range(len(cur_solution) - 1):
        for j in range(i + 1, len(cur_solution)):
            value += dist[i][j] * flow[cur_solution[i]][cur_solution[j]]
    value *= 2
    return value

In [2]:
def swap_factories(solution: list, i: int, j: int):
    sol_copy = solution.copy()
    tmp = sol_copy[i]
    sol_copy[i] = sol_copy[j]
    sol_copy[j] = tmp
    return sol_copy

In [9]:
def local_search(n_factories: int, dist: list, flow: list):
    '''Local search with first-improvement + don't look bits'''
    import time
    start = time.time()
    
    best_sol = [i for i in range(n_factories)]
    best_sol_value = get_flow_value(best_sol, dist, flow)
    while True:
        dlb = [False for i in range(n_factories)]
        improvement_flag = False
        for i in range(n_factories):
            dlb_flag = True
            for j in range(n_factories):
                if not dlb[j] and i != j:
                    new_sol = swap_factories(best_sol, i, j)
                    delta = 0
                    for k in range(n_factories):
                        if k != i and k != j:
                            delta -= dist[i][k] * flow[best_sol[i]][best_sol[k]]
                            delta -= dist[j][k] * flow[best_sol[j]][best_sol[k]]
                            delta += dist[i][k] * flow[new_sol[i]][new_sol[k]]
                            delta += dist[j][k] * flow[new_sol[j]][new_sol[k]]
                    delta *= 2
                    if delta > 0:
                        dlb_flag = False
                        improvement_flag = True
                        best_sol_value += delta
                        best_sol = new_sol
            if dlb_flag:
                dlb[i] = True
        if not improvement_flag:
            break
        
        # print(best_sol, best_sol_value, '\n')
    end = time.time()    
    
    return best_sol, best_sol_value, end - start

In [12]:
data = []

In [14]:
files_list = ['tai20a.txt', 'tai40a.txt', 'tai60a.txt', 'tai80a.txt', 'tai100a.txt']
 
for file in files_list:       
    with open(f'Lab4/benchmarks/{file}', 'r') as f:
        dist = []
        flow = []
        n_factories = int(f.readline())
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            dist.append(arr)
        
        f.readline()
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            flow.append(arr)
        
        best_sol, best_sol_value, time = local_search(n_factories, dist, flow)
        data.append([file[:-4] , best_sol_value, time])
        print(f'Best solution: {best_sol}')
        print(f'Best solution flow value: {best_sol_value}')
        print(f'Time: {time}\n')
        with open(f'Lab4/ls_outputs/{file}', 'w') as output_file:
            print(*best_sol, file=output_file)

Best solution: [3, 11, 9, 5, 15, 16, 6, 7, 14, 2, 8, 17, 18, 1, 19, 0, 13, 4, 12, 10]
Best solution flow value: 1043396
Time: 0.009002208709716797

Best solution: [26, 2, 18, 5, 24, 8, 39, 1, 19, 7, 29, 32, 3, 28, 11, 13, 25, 38, 27, 12, 9, 20, 15, 14, 36, 16, 0, 37, 6, 21, 17, 31, 22, 30, 34, 35, 10, 4, 33, 23]
Best solution flow value: 4305536
Time: 0.14799880981445312

Best solution: [40, 8, 21, 14, 27, 17, 1, 35, 34, 0, 44, 59, 30, 2, 13, 16, 58, 6, 57, 3, 54, 4, 7, 22, 37, 10, 12, 25, 26, 23, 20, 18, 36, 33, 32, 11, 46, 19, 38, 5, 41, 31, 42, 43, 29, 39, 56, 51, 9, 48, 50, 28, 52, 53, 47, 55, 45, 24, 49, 15]
Best solution flow value: 9447574
Time: 0.36550021171569824

Best solution: [64, 24, 33, 4, 10, 40, 56, 46, 3, 61, 39, 27, 57, 73, 8, 50, 18, 74, 23, 19, 22, 28, 1, 17, 15, 49, 12, 6, 21, 0, 30, 71, 38, 52, 29, 14, 37, 36, 63, 42, 79, 77, 26, 2, 72, 45, 43, 48, 9, 32, 69, 51, 35, 53, 54, 76, 7, 41, 34, 44, 60, 16, 11, 31, 59, 65, 66, 58, 68, 55, 70, 25, 20, 13, 47, 75, 67, 62,

In [20]:
import pandas as pd

df = pd.DataFrame(data, columns=['sample', 'LS best value', 'LS time'])
df

,sample,LS best value,LS time
0,tai20a,1043396,0.009002
1,tai40a,4305536,0.147999
2,tai60a,9447574,0.365500
3,tai80a,17250548,0.870500
4,tai100a,26188816,1.049999
